
# Spotify Churn Prediction — Detailed Notebook

**Contents:** EDA → Preprocessing → Feature Engineering → Modeling → SMOTE & Imbalance Handling → Threshold Tuning → Results & Conclusions

This notebook is detailed (code + explanations) and intended to be run end-to-end. It expects the dataset file `spotify_churn_dataset.csv` to be in the same folder (or update the `data_path` variable).


In [ ]:

# Imports and settings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, precision_recall_curve, RocCurveDisplay
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
plt.style.use('seaborn-whitegrid')
data_path = 'spotify_churn_dataset.csv'  # update if necessary
output_dir = Path('notebook_outputs')
output_dir.mkdir(exist_ok=True)


In [ ]:

# Load dataset and inspect
df = pd.read_csv(data_path)
print('Shape:', df.shape)
df.head()


In [ ]:

# Basic EDA
print(df.info())
print('\nMissing values per column:\n', df.isnull().sum())
print('\nTarget distribution (is_churned):\n', df['is_churned'].value_counts(normalize=True))

# Quick visualizations
plt.figure(figsize=(6,4))
df['is_churned'].value_counts().plot(kind='bar', color=['#1DB954','#191414'])
plt.title('Churn vs Active (counts)')
plt.xticks([0,1], ['Active (0)','Churned (1)'])
plt.ylabel('Count')
plt.savefig(output_dir/'churn_counts.png')
plt.show()


In [ ]:

# Feature engineering
df = df.copy()
df['engagement_score'] = df['songs_played_per_day'] * df['listening_time']
df['listening_time_hours'] = df['listening_time'].replace(0, np.nan)
df['engagement_per_hour'] = df['engagement_score'] / df['listening_time_hours']
df['engagement_per_hour'] = df['engagement_per_hour'].fillna(df['engagement_score'])
df['high_skip'] = (df['skip_rate'] > df['skip_rate'].median()).astype(int)
bins = [0,17,25,35,50,100]
labels = ['<18','18-25','26-35','36-50','50+']
df['age_group'] = pd.cut(df['age'], bins=bins, labels=labels, right=True)
df['mobile_and_offline'] = ((df['device_type']=='Mobile') & (df['offline_listening']==1)).astype(int)

# Show new features
df[['engagement_score','engagement_per_hour','high_skip','age_group','mobile_and_offline']].head()


In [ ]:

# Select features and target
features = ['gender','age','age_group','country','subscription_type','listening_time',
            'songs_played_per_day','skip_rate','device_type','ads_listened_per_week',
            'offline_listening','engagement_score','engagement_per_hour','high_skip','mobile_and_offline']
target = 'is_churned'
X = df[features].copy()
y = df[target].copy()

# Identify numeric & categorical
numeric_features = ['age','listening_time','songs_played_per_day','skip_rate','ads_listened_per_week','offline_listening','engagement_score','engagement_per_hour','high_skip','mobile_and_offline']
categorical_features = ['gender','age_group','country','subscription_type','device_type']

# Pipelines
numeric_transformer = Pipeline([('imputer', SimpleImputer(strategy='median')), ('scaler', StandardScaler())])
categorical_transformer = Pipeline([('imputer', SimpleImputer(strategy='constant', fill_value='missing')), ('onehot', OneHotEncoder(handle_unknown='ignore', sparse=False))])
preprocessor = ColumnTransformer([('num', numeric_transformer, numeric_features), ('cat', categorical_transformer, categorical_features)])

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)
print('Train shape:', X_train.shape, 'Test shape:', X_test.shape)


In [ ]:

# Baseline models
models = {
    'LogisticRegression': Pipeline([('pre', preprocessor), ('clf', LogisticRegression(max_iter=1000))]),
    'DecisionTree': Pipeline([('pre', preprocessor), ('clf', DecisionTreeClassifier(random_state=42))]),
    'RandomForest': Pipeline([('pre', preprocessor), ('clf', RandomForestClassifier(n_estimators=150, random_state=42))]),
    'GradientBoosting': Pipeline([('pre', preprocessor), ('clf', GradientBoostingClassifier(random_state=42))])
}

results = []
for name, pipeline in models.items():
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    y_proba = pipeline.predict_proba(X_test)[:,1] if hasattr(pipeline.named_steps['clf'], 'predict_proba') else None
    results.append({
        'model': name,
        'accuracy': round(accuracy_score(y_test, y_pred),4),
        'precision': round(precision_score(y_test, y_pred, zero_division=0),4),
        'recall': round(recall_score(y_test, y_pred, zero_division=0),4),
        'f1': round(f1_score(y_test, y_pred, zero_division=0),4),
        'roc_auc': round(roc_auc_score(y_test, y_proba),4) if y_proba is not None else None
    })

pd.DataFrame(results).sort_values('f1', ascending=False)


In [ ]:

# Method A: class_weight balanced
from sklearn.pipeline import Pipeline as SklearnPipeline
rf_balanced = SklearnPipeline([('pre', preprocessor), ('clf', RandomForestClassifier(n_estimators=150, random_state=42, class_weight='balanced'))])
rf_balanced.fit(X_train, y_train)
y_pred_bal = rf_balanced.predict(X_test)
y_proba_bal = rf_balanced.predict_proba(X_test)[:,1]
metrics_bal = {
    'accuracy': round(accuracy_score(y_test, y_pred_bal),4),
    'precision': round(precision_score(y_test, y_pred_bal, zero_division=0),4),
    'recall': round(recall_score(y_test, y_pred_bal, zero_division=0),4),
    'f1': round(f1_score(y_test, y_pred_bal, zero_division=0),4),
    'roc_auc': round(roc_auc_score(y_test, y_proba_bal),4)
}
print('Class-weighted RF metrics:', metrics_bal)

# Method B: SMOTE oversampling
smote = SMOTE(random_state=42)
rf_smote = ImbPipeline([('pre', preprocessor), ('smote', smote), ('clf', RandomForestClassifier(n_estimators=150, random_state=42))])
rf_smote.fit(X_train, y_train)
y_pred_smote = rf_smote.predict(X_test)
y_proba_smote = rf_smote.predict_proba(X_test)[:,1]
metrics_smote = {
    'accuracy': round(accuracy_score(y_test, y_pred_smote),4),
    'precision': round(precision_score(y_test, y_pred_smote, zero_division=0),4),
    'recall': round(recall_score(y_test, y_pred_smote, zero_division=0),4),
    'f1': round(f1_score(y_test, y_pred_smote, zero_division=0),4),
    'roc_auc': round(roc_auc_score(y_test, y_proba_smote),4)
}
print('SMOTE RF metrics:', metrics_smote)


In [ ]:

# Threshold tuning on SMOTE model
y_proba = rf_smote.predict_proba(X_test)[:,1]
precisions, recalls, thresholds = precision_recall_curve(y_test, y_proba)
f1_scores = 2 * (precisions * recalls) / (precisions + recalls + 1e-8)

best_idx = np.argmax(f1_scores)
best_threshold = thresholds[best_idx]
best_f1 = f1_scores[best_idx]

print('Best threshold:', best_threshold)
print('Best F1:', best_f1)

# Metrics at best threshold
y_pred_tuned = (y_proba >= best_threshold).astype(int)
metrics_tuned = {
    'threshold': round(best_threshold,4),
    'accuracy': round(accuracy_score(y_test, y_pred_tuned),4),
    'precision': round(precision_score(y_test, y_pred_tuned, zero_division=0),4),
    'recall': round(recall_score(y_test, y_pred_tuned, zero_division=0),4),
    'f1': round(f1_score(y_test, y_pred_tuned, zero_division=0),4),
    'roc_auc': round(roc_auc_score(y_test, y_proba),4)
}
print('Metrics at tuned threshold:', metrics_tuned)

# Save PR/F1 vs threshold plot
import matplotlib.pyplot as plt
plt.figure(figsize=(8,5))
plt.plot(thresholds, precisions[:-1], label='Precision')
plt.plot(thresholds, recalls[:-1], label='Recall')
plt.plot(thresholds, f1_scores[:-1], label='F1')
plt.axvline(best_threshold, color='red', linestyle='--', label=f'Best thr={best_threshold:.2f}')
plt.xlabel('Threshold')
plt.ylabel('Score')
plt.title('Precision / Recall / F1 vs Threshold (SMOTE RF)')
plt.legend()
plt.savefig(output_dir/'threshold_prf1.png')
plt.show()


In [ ]:

# Feature importance from RF (after preprocessing): extract names
ohe = rf_smote.named_steps['pre'].named_transformers_['cat'].named_steps['onehot']
cat_cols = list(ohe.get_feature_names_out(categorical_features))
feature_names = numeric_features + cat_cols
importances = rf_smote.named_steps['clf'].feature_importances_
feat_imp = pd.DataFrame({'feature': feature_names, 'importance': importances}).sort_values('importance', ascending=False).reset_index(drop=True)
feat_imp.head(20).to_csv(output_dir/'feature_importances.csv', index=False)
feat_imp.head(20)


In [ ]:

# Save key artifacts
pd.DataFrame([metrics_bal]).to_csv(output_dir/'rf_balanced_metrics.csv', index=False)
pd.DataFrame([metrics_smote]).to_csv(output_dir/'rf_smote_metrics.csv', index=False)
pd.DataFrame([metrics_tuned]).to_csv(output_dir/'rf_smote_threshold_metrics.csv', index=False)
print('Artifacts saved to', output_dir.resolve())



## Conclusion & Next Steps

- SMOTE + threshold tuning provided the best practical performance for detecting churn (high recall).
- Next steps: try XGBoost/LightGBM, do deeper feature engineering (session recency), and use SHAP for interpretability.
- Prepare a GitHub repo with this notebook, the final report, slides, and outputs for submission.
